In [20]:
from langchain_core.messages import AIMessage, HumanMessage
from pprint import pprint
import os
from dotenv import load_dotenv

load_dotenv()
openai_api_key    = os.getenv("OPENAI_API_KEY")
gemini_api_key    = os.getenv("GEMINI_API_KEY")
groq_api_key      = os.getenv("GROQ_API_KEY")
langchain_api_key = os.getenv("LANGCHAIN_API_KEY")

In [21]:
### im Considering to make a tool to Provide to to the llm so that it can now work on the basis of the Tool Defined Rather than then its own Knowledge base 

def tool1(a:int,b:int)-> int:
    """ Add a and b 
    Args:
    a ( int ) --> first Number 
    b ( int ) --> Second Number
    You need to Add the 2 Numbers and Say (Hey ishn, i will do whatever you Say!)
    """

    return a+b

In [22]:
from langchain_groq import ChatGroq
llm = ChatGroq(model="qwen/qwen3-32b")
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000023D61CBBE50>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000023D61561B90>, model_name='qwen/qwen3-32b', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [23]:
llm_with_tools = llm.bind_tools([tool1])
tool_call = llm_with_tools.invoke([HumanMessage(content="What is 1+4")])
tool_call.tool_calls

[{'name': 'tool1',
  'args': {'a': 1, 'b': 4},
  'id': 'gdpvwbq0g',
  'type': 'tool_call'}]

In [24]:
from typing_extensions import TypedDict
from typing import Annotated
from langchain_core.messages import AnyMessage
from langgraph.graph.message import add_messages

In [25]:
class State(TypedDict):
    messages:Annotated[list[AnyMessage],add_messages]

In [26]:
initial_messages = [AIMessage(content="How May i Help ? ",name = "Ai")]
initial_messages.append(HumanMessage(content="i want to Learn Coding , How to Start ",name="ishn"))
initial_messages

[AIMessage(content='How May i Help ? ', additional_kwargs={}, response_metadata={}, name='Ai'),
 HumanMessage(content='i want to Learn Coding , How to Start ', additional_kwargs={}, response_metadata={}, name='ishn')]

In [27]:
llm_message = AIMessage(content="which Coding Language You Want to Start with >? ",name="Ai")
llm_message

AIMessage(content='which Coding Language You Want to Start with >? ', additional_kwargs={}, response_metadata={}, name='Ai')

In [28]:
all_messages_till_now = add_messages(initial_messages, llm_message)
all_messages_till_now

[AIMessage(content='How May i Help ? ', additional_kwargs={}, response_metadata={}, name='Ai', id='d7e6f7c1-7bc4-40ee-8de4-ce5140db4ac4'),
 HumanMessage(content='i want to Learn Coding , How to Start ', additional_kwargs={}, response_metadata={}, name='ishn', id='c97281ab-626a-4af1-ab32-ba25de86f933'),
 AIMessage(content='which Coding Language You Want to Start with >? ', additional_kwargs={}, response_metadata={}, name='Ai', id='9a1f6e45-df19-4e1b-a6d3-4799f7bde185')]

In [29]:
def llm_tool(state:State):
    return {"messages":[llm_with_tools.invoke(state["messages"])]}

In [33]:
from langgraph.graph import StateGraph, START, END

Graph_builder = StateGraph(State)
Graph_builder.add_node("llm_tool",llm_tool)
Graph_builder.add_edge(START,"llm_tool")
Graph_builder.add_edge("llm_tool",END)

In [39]:
graph = Graph_builder.compile()
messages = graph.invoke({"messages":"What is 2+2"})


for message in messages["messages"]:
    message.pretty_print()

================================ Human Message =================================

What is 2+2
================================== Ai Message ==================================
Tool Calls:
  tool1 (b8522p4zg)
 Call ID: b8522p4zg
  Args:
    a: 2
    b: 2


In [40]:
tools = [tool1]
from langgraph.prebuilt import ToolNode
from langgraph.prebuilt import tools_condition

In [41]:
builder=StateGraph(State)

## Add nodes

builder.add_node("llm_tool",llm_tool)
builder.add_node("tools",ToolNode(tools))

In [42]:
builder.add_edge(START,"llm_tool")
builder.add_conditional_edges(
    "llm_tool",
    # If the latest message (result) from assistant is a tool call -> tools_condition routes to tools
    # If the latest message (result) from assistant is a not a tool call -> tools_condition routes to END
    tools_condition
)
builder.add_edge("tools",END)


graph_builder = builder.compile()

In [44]:
messages=graph.invoke({"messages":"What is 7 plus 11110"})

for message in messages["messages"]:
    message.pretty_print()

================================ Human Message =================================

What is 7 plus 11110
================================== Ai Message ==================================
Tool Calls:
  tool1 (0ke3zqtg5)
 Call ID: 0ke3zqtg5
  Args:
    a: 7
    b: 11110


Adding a node to a graph that has already been compiled. This will not be reflected in the compiled graph.
Adding an edge to a graph that has already been compiled. This will not be reflected in the compiled graph.
Adding an edge to a graph that has already been compiled. This will not be reflected in the compiled graph.


================================ Human Message =================================

What is 7 plus 11110
================================== Ai Message ==================================
Tool Calls:
  tool1 (8csecj1xd)
 Call ID: 8csecj1xd
  Args:
    a: 7
    b: 11110
